<a href="https://colab.research.google.com/github/CAIOZANETTI/estimar_custo/blob/main/estimativa_custo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Carregar

In [45]:
# @title 1.1.Pacotes Python
# Instalar

import os
import pandas as pd

# configuração do Notebook
from IPython.display import display, Markdown
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_colwidth', None) # mostrar nome completo

import warnings
# Suprimir todos os avisos
warnings.filterwarnings("ignore")


In [46]:
# @title 1.2.Carregar Github
!git clone https://github.com/CAIOZANETTI/estimar_custo


fatal: destination path 'estimar_custo' already exists and is not an empty directory.


In [47]:
# @title 1.3.Nomear Funções
import estimar_custo.funcoes as fx

In [48]:
# @title 1.4.Criar Dicionarios "dfs={}"
# natural, normalizado, classificado, transformado

dfs = {
    'natural':{'grupos':0,'plan':0,'cpu':0,'ins':0},
    'normalizado':{'grupos':0,'plan':0,'cpu':0,'ins':0},
    'plan_cpu_ins':{'natural':0,'normalizado':0,'apelidado':0},
}
# apelidade, reconhecido, desconhecido...
dfs

{'natural': {'grupos': 0, 'plan': 0, 'cpu': 0, 'ins': 0},
 'normalizado': {'grupos': 0, 'plan': 0, 'cpu': 0, 'ins': 0},
 'plan_cpu_ins': {'natural': 0, 'normalizado': 0, 'apelidado': 0}}

In [49]:
# @title 1.5.Criar Diretórios
!mkdir relatorio
!mkdir planilha
!mkdir custo

mkdir: cannot create directory ‘relatorio’: File exists
mkdir: cannot create directory ‘planilha’: File exists
mkdir: cannot create directory ‘custo’: File exists


#2.Importar

In [50]:
# @title 2.1.Planilha Apelidos e Grupos
url ='https://docs.google.com/spreadsheets/d/1MueS4TbXhVEcX5coDPk2ty0cK3scMDSBNtI2S58isWg/export?format=csv'
df = pd.read_csv(url)

df.fillna(0, inplace=True)
display(df.info())
dfs['natural']['grupos'] = df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   disciplina       617 non-null    object
 1   etapa            617 non-null    object
 2    grupo           617 non-null    object
 3    subgrupo        617 non-null    object
 4    tipo            617 non-null    object
 5   unid             617 non-null    object
 6   a_contem         617 non-null    object
 7   b_contem         617 non-null    object
 8   c_contem         617 non-null    object
 9   d_contem         617 non-null    object
 10  ignorar          617 non-null    object
 11  fx_apelido       617 non-null    object
 12  apelido_nomeado  617 non-null    object
 13  apelido          617 non-null    object
dtypes: object(14)
memory usage: 67.6+ KB


None

In [51]:
# @title 2.2. Escolher Planilha **ler_pandas**
diretorio = '/content/planilha'
files = os.listdir('/content/planilha')
arquivo = diretorio+'/'+files[0]

df=pd.read_excel(arquivo, sheet_name='ler_pandas')

display(df.info())
dfs['natural']['plan'] = df

IsADirectoryError: [Errno 21] Is a directory: '/content/planilha/.ipynb_checkpoints'

In [ ]:
# @title 2.3. Escolher planilha **Custo** { form-width: "25%" }
diretorio = '/content/planilha'
files = os.listdir('/content/custo')
arquivo = diretorio+'/'+files[0]

dfs['natural']['cpu']=pd.read_excel(arquivo, sheet_name='cpu',usecols='A:F',skiprows=1)
dfs['natural']['ins']=pd.read_excel(arquivo, sheet_name='ins',usecols='A:J',skiprows=1)

display(dfs['natural']['cpu'].info())

In [ ]:
# @title 2.4. Mostrar Planilha **ins**
dfs['natural']['ins']=pd.read_excel(arquivo, sheet_name='ins',usecols='A:J',skiprows=1)
display(dfs['natural']['ins'].info())

# 3.Normalizar

In [ ]:
# @title 3.1.CPU
df = dfs['natural']['cpu']

display(Markdown('natural'))
display(df.shape)
display(df.head(3))
df['unit'] = df['mat']+df['mo']
df['parcial'] = df['qtd']*df['unit']

df['origem'] = 'custo'
df['obra'] = obra
df['arquivo'] = arquivo
df['aba'] = 'cpu'

df.drop(['mat', 'mo'], axis=1, inplace=True)
display(Markdown('normalizado'))
display(df.head(3))

dfs['normalizado']['cpu'] = df

In [ ]:
# @title 3.2.**Insumos**
df = dfs['natural']['ins']
display(Markdown('natural'))
display(df.shape)
display(df.head(3))

df['origem'] = 'custo'
df['obra'] = obra
df['arquivo'] = arquivo
df['aba'] = 'ins'

df.drop(['grupo', 'subgrupo','fd'], axis=1, inplace=True)
display(Markdown('Normalizado'))
display(df.head(3))
dfs['normalizado']['ins'] = df

In [ ]:
# @title 3.3. Unir (plan+ins+cpu)
df = pd.concat([
    dfs['natural']['plan'],
    dfs['normalizado']['cpu'],
    dfs['normalizado']['ins']
])

display(df.shape)
display(df.head(3))
dfs['plan_cpu_ins']['natural']=df

In [ ]:
# @title 3.4. Unidades
df = dfs['plan_cpu_ins']['natural']

df = fx.converter_unidades(df)
display(Markdown('Normalizado'))
display(" ".join(list(df['unid'].unique())))
dfs['plan_cpu_ins']['normalizado'] = df

# 4.Apelidar

In [ ]:
# @title 4.4.Conhecidos
df = dfs['plan_cpu_ins']['normalizado']
df1 = dfs['natural']['grupos']
#display(df1.shape)
df = fx.nome_minusculo(df)
df2 = fx.agrupar_df(df,df1)

pd.set_option('display.max_colwidth', 20) # mostrar nome completo
display(df1.shape)
display(df1.head(3))
pd.set_option('display.max_colwidth', None)

## 4.5.Desconhecidos

In [ ]:
# @title 4.5.1.Geral
df_nan = df1[df1['qtd']>0]
df_nan = df_nan[df_nan['apelido'].isna()]

display(df_nan[['id','unid','nome','origem','aba']].head(5))

pivot= df_nan.pivot_table(index=['origem','aba'], values=['nome'],aggfunc='count',margins=True)
display(Markdown(3*"--"))
display(Markdown("**Desconhecidos**: "+str(df_nan.shape[0])))
display(pivot.transpose())

In [ ]:
# @title 4.5.2.Insumos
df_ins_desconecidos = df_nan[(df_nan['origem']=='custo') & (df_nan['aba']=='ins')]
display(df_ins_desconecidos[['nome','unid']])

In [ ]:
# @title 4.5.3.CPUs
df_cpu_desconecidos = df_nan[(df_nan['origem']=='custo') & (df_nan['aba']=='cpu')]
display(df_cpu_desconecidos[['nome','unid']])

# 5.Pivots

In [ ]:
# @title 5.1.Planilha
df2 = df1[df1['origem']=='cliente']

pivot_unid_apelido_nome= df2.pivot_table(index=['unid','apelido','nome',],
                              values=['qtd'],#values=['unit','qtd', 'parcial'],
                              aggfunc='sum')

pd.set_option('display.max_rows', None)
display(pivot_unid_apelido_nome)

### Cpu

In [ ]:
# @title 5.2.CPU

df2 = df1[(df1['origem']=='custo') & (df1['aba']=='cpu')]

pivot_unid_apelido_nome= df2.pivot_table(index=['unid','apelido','nome',],
                              values=['qtd'],#values=['unit','qtd', 'parcial'],
                              aggfunc='sum')

pd.set_option('display.max_rows', None)
display(df2.shape)
display(pivot_unid_apelido_nome)

# 6.Salvar

In [ ]:
# @title 6.1.Planilha
salvar = 'plan_cliente//03_relatorio//'+arquivo_nome+'_plan_agrupada.xlsx'

#display(df1)
with pd.ExcelWriter(salvar) as writer:
    # Save each DataFrame to a specific sheet
    df1.to_excel(writer, sheet_name='00_plan', index=False)
    pivot_unid_apelido_nome.to_excel(writer, sheet_name='01_unid_apelido_nome', index=True,merge_cells=False)

display(salvar)

# 7.Validar

### Apelidos Usados

In [ ]:
# @title 7.1.Planilha
#filtro somente da planilha!!!
df2 = df1[df1['origem']=='cliente']

apelidos_usados_plan=df2['apelido'].unique()
display('apelidos encontrados plan cliente: '+ str(len(apelidos_usados_plan)))

In [ ]:
# @title 7.2.Planilha
#filtro somente da planilha!!!
df2 = df1[df1['origem']=='custo']

apelidos_usados_cpu=df2['apelido'].unique()
display('apelidos encontrados plan custo: '+ str(len(apelidos_usados_cpu)))

### CPU

In [ ]:
# @title 6.1.CPU
df2 = df1[(df1['origem']=='custo') & (df1['aba']=='cpu')]
apelidos_cpu=df2['apelido'].unique()
display(apelidos_cpu)
pivot_unid_apelido_nome= df2.pivot_table(index=['unid','apelido','nome',],
                              values=['qtd'],#values=['unit','qtd', 'parcial'],
                              aggfunc='sum')
display(pivot_unid_apelido_nome)

### Custo

In [ ]:
# @title 6.1.CPU
apelidos_usados_plan

df1 =df1[df1['apelido'].isin(apelidos_usados_plan)]
df2 =df1[(df1['apelido'].notnull())&(df1['aba']!='ins')]
pivot2= df2.pivot_table(index=['unid','apelido','origem','nome','unit'],
                              values=['qtd'],#values=['unit','qtd', 'parcial'],
                              aggfunc='sum')
display(pivot2)